In [1]:
import or_gym
from or_gym.utils import create_env
import gym
from gym import spaces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
env_name = 'InvManagement-v2'
env = or_gym.make(env_name)

In [3]:
# Run sample episode
env.reset()
while True:
    action = env.sample_action()
    out = env.step(action)
    if out[2]:
        break

In [4]:
import ray
from ray.rllib import agents
from ray import tune
ray.init(ignore_reinit_error=True)

Instructions for updating:
non-resource variables are not supported in the long term


2020-11-04 12:20:39,907	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.104',
 'raylet_ip_address': '192.168.1.104',
 'redis_address': '192.168.1.104:6379',
 'object_store_address': '/tmp/ray/session_2020-11-04_12-20-39_404104_10600/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-04_12-20-39_404104_10600/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-11-04_12-20-39_404104_10600',
 'metrics_export_port': 49454}

In [5]:
from ray.rllib.models.tf.tf_modelv2 import TFModelV2

In [7]:
agents.ppo.DEFAULT_CONFIG

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 4000,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  '_time_major': False,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_action_dist': None,
  'custom_preprocessor': None},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {},
 'env': None,
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preprocessor_pref': 'deepmind',
 'lr': 5e-05,
 'monitor': False,
 'log_level': 'WARN',
 'callbacks': ra

In [5]:
def register_env(env_name, env_config={}):
    env = create_env(env_name)
    tune.register_env(env_name, 
        lambda env_name: env(env_name,
            env_config=env_config))
    
register_env(env_name)

In [8]:
env_name = 'InvManagement-v2'
register_env(env_name1)
# trainer = agents.ppo.PPOTrainer(env=env_name1)
# trainer.train()

In [17]:
env_config = {}
rl_config = dict(
    env=env_name,
    num_workers=2,
    env_config=env_config,
    model=dict(
        vf_share_layers=False,
        fcnet_activation='elu',
        fcnet_hiddens=[256, 256]
    ),
    lr=1e-6
)

ray.init(ignore_reinit_error=True)
agent = agents.ppo.PPOTrainer(env=env_name,
    config=rl_config)
results = []
for i in range(500):
    res = agent.train()
    results.append(res)
    if (i+1) % 5 == 0:
        print('\rIter: {}\tReward: {:.2f}'.format(
                i+1, res['episode_reward_mean']), end='')
ray.shutdown()

2020-11-04 06:51:52,263	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.
(pid=9755) WARNING:tensorflow:From /home/christian/anaconda3/envs/or-gym-dev/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=9755) Instructions for updating:
(pid=9755) non-resource variables are not supported in the long term
(pid=9746) WARNING:tensorflow:From /home/christian/anaconda3/envs/or-gym-dev/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=9746) Instructions for updating:
(pid=9746) non-resource variables are not supported in the long term
2020-11-04 06:51:55,032	WARNING util.py:39 -- Install gputil for GPU system monitoring.
(pid=9755) WARNING:tensorflow:From /home/chris

Iter: 180	Reward: 297.74

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Unpack values from each iteration
rewards = np.hstack([i['hist_stats']['episode_reward'] 
    for i in results])
pol_loss = [
    i['info']['learner']['default_policy']['policy_loss'] 
    for i in results]
vf_loss = [
    i['info']['learner']['default_policy']['vf_loss'] 
    for i in results]
p = 100
mean_rewards = np.array([np.mean(rewards[i-p:i+1]) 
                if i >= p else np.mean(rewards[:i+1]) 
                for i, _ in enumerate(rewards)])
std_rewards = np.array([np.std(rewards[i-p:i+1])
               if i >= p else np.std(rewards[:i+1])
               for i, _ in enumerate(rewards)])

fig = plt.figure(constrained_layout=True, figsize=(20, 10))
gs = fig.add_gridspec(2, 4)
ax0 = fig.add_subplot(gs[:, :-2])
ax0.fill_between(np.arange(len(mean_rewards)), 
                 mean_rewards - std_rewards, 
                 mean_rewards + std_rewards, 
                 label='Standard Deviation', alpha=0.3)
ax0.plot(mean_rewards, label='Mean Rewards')
ax0.set_ylabel('Rewards')
ax0.set_xlabel('Episode')
ax0.set_title('Training Rewards')
ax0.legend()

ax1 = fig.add_subplot(gs[0, 2:])
ax1.plot(pol_loss)
ax1.set_ylabel('Loss')
ax1.set_xlabel('Iteration')
ax1.set_title('Policy Loss')

ax2 = fig.add_subplot(gs[1, 2:])
ax2.plot(vf_loss)
ax2.set_ylabel('Loss')
ax2.set_xlabel('Iteration')
ax2.set_title('Value Function Loss')
plt.show()